# Дашборд Кейса №2

## Импорты библиотек

In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr  # new
import os  # new

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)

sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

## Вспомогательные функции

Подгрузка csv файла

In [90]:
def load_data(file):
    if file is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате csv", None
    df = pd.read_csv(
        file.name, parse_dates=["survey_creation_dt", "survey_response_dt"]
    )
    filename = os.path.basename(file.name)
    text_status = "Файл " + str(filename) + " успешно загружен"
    text_description = (
        "Файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description

Очистка данных

In [ ]:
def clear_data(df):
    if df is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате CSV.", None

    # 1. Вычисление времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    # 2. Исправление перепутанных дат (где ответ раньше создания)
    negative_before = (df["response_time_days"] < 0).sum()
    text_description = f"Записей с отрицательным временем ответа: {negative_before}.\n"

    df["dates_swapped"] = df["response_time_days"] < 0
    swap = df["dates_swapped"]
    df.loc[swap, ["survey_creation_dt", "survey_response_dt"]] = df.loc[
        swap, ["survey_response_dt", "survey_creation_dt"]
    ].values

    # Пересчёт времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    negative_after = (df["response_time_days"] < 0).sum()
    text_description = text_description + (f"После исправления: {negative_after}. \n")

    # 3. Заполнение пропусков в language
    language_missing = df["language"].isnull().sum()
    df["language"] = df["language"].fillna("RU")
    text_description = text_description + (
        f"Заполнено пропусков в language: {language_missing}. \n"
    )

    text_status = "Файл успешно обработан"
    text_description = text_description + (
        "Актуальный файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description


def how_many_nulls(df):
    mapping = {
        "survey_creation"
    }

## Структура дашборда

In [111]:
# структура Дашборда
with gr.Blocks(title="Кейс 2 - Команда 8") as case2:
    gr.Markdown("""
    <div style="text-align:center">

    # Дашборд команды №8  
    Кейс №2. Пенсионный фонд, предоставляющий клиентам цифровые сервисы.

    </div>
    """)

    # df_state для того, чтобы сохранить данные для кнопок
    df_state = gr.State(None)

    with gr.Tabs():
        # структура дашборда

        # Таб с загрузкой данных
        with gr.Tab("Загрузка данных"):
            with gr.Row():
                result_box = gr.Textbox(label="Статус", lines = 3, max_lines= 15, scale = 1)
                description_box = gr.Textbox(label="Описание", lines = 3, max_lines = 15, scale = 3)
            file_input = gr.File(label="Выберите CSV-файл", file_types=[".csv"], scale = 1, height = 100)
            with gr.Row():
                load_button = gr.Button("Загрузить файл csv")
                clear_button = gr.Button("Очистить данные")
            head_df = gr.DataFrame(label="Пример данных", type="pandas")

    # операции

    # Загрузка CSV
    load_button.click(
        load_data,
        inputs=[file_input],
        outputs=[df_state, head_df, result_box, description_box],
    )
    # Очистка данных
    clear_button.click(
        clear_data,
        inputs=[df_state],
        outputs=[df_state, head_df, result_box, description_box],
    )

In [112]:
case2.launch(share=False, inline=True)

* Running on local URL:  http://127.0.0.1:7900
* To create a public link, set `share=True` in `launch()`.
